# Optimisation bayésienne d'un simulateur coûteux


Le but du TP est d'optimiser une fonction coûteuse de type "boîte noire" en utilisant une technique d'optimisation bayésienne. On considère ici un simulateur jouet correspondant à la modélisation d'une catapulte.
Dans cet exemple, il y a 4 variables d'entrée et 1 sortie correspondant à la distance du point d'impact du projectile. On veut trouver la (ou les) valeur(s) du (des) variable(s) d'entrée correspondant à la distance maximale. A noter que le temps de calcul est immédiat, mais on considérera, à titre pédagogique, que le budget de calcul est limité à seulement 36 simulations, réparties en 16 points initiaux + 20 points supplémentaires.

1. Optimisation à la main
On donne ici une application shiny (auteur Nicolas Durrande), qui permet de visualiser les simulations de façon interactive.

In [3]:
library(shiny)
runApp()


Listening on http://127.0.0.1:5506


Question: En jouant manuellement sur les réglages des 4 variables d'entrée, proposer un réglage optimal (permettant de maximiser la distance).

Pour poursuivre la lecture du notebook, il faut interrompre le noyau (bouton carré !) et charger les fonctions utiles 

In [4]:
source("catapultSettings.R")
source("catapultFunctions.R")

[1] 4


2. Commençons par créer un plan d'expériences initial et calculons les valeurs correspondantes.

In [ ]:
library(DiceDesign)
X0 <- lhsDesign(n = 16, dimension = 4)$design
Xopt <- maximinESE_LHS(X0, it=10)
## you may be interested in the convergence
#plot(Xopt$critValues,type="l")
X <- Xopt$design
colnames(X) <- c("axe", "butee", "ressort1", "ressort2")
pairs(X)
## compute the output values
Y <- apply(X, 1, runExperiment)[1, ]

Question : Observer que le plan est de type "space-filling" (i.e. remplit l'espace). <br> Pourquoi l'a t-on choisi de cette forme ? Quel est le maximum courant ? Est-il éloigné du maximum calculé à la main ?

3. Statistiques descriptives. Voit-on une relation simple entre entrées et sortie ? Quelle intuition peut-on avoir sur la région où se trouve l'optimum ?

In [ ]:
pairs(cbind(Y, X))

4. Métamodèle de régression. Tentons un modèle linéaire.
Question : Quel optimum est suggéré par le (les) modèle(s) obtenu(s) ? Que vaut la distance en ce point ? Cela confirme t-il les observations précédentes ?

In [ ]:
myData <- data.frame(X, Y = Y)
m <- lm(Y ~ ., data = myData)
summary(m)
m2 <- lm(Y~. + I(axe^2) + I (butee^2) + I (ressort1^2) + I(ressort2^2), data = myData)
summary(m2)
mstep <- step(object = m, scope = m2, direction = "both", k = log(length(Y)))
summary(mstep)

5. Maintenant, utilisons la méthode EGO (optimisation bayésienne)
Question : rappeler ses principes.

In [ ]:
library(DiceKriging)
m0 <- km(~1, design=X, response=Y)
print(m0)   # display model
plot(m0)    # visual model validation

Question : Que dire de la qualité de ce premier modèle ? Commenter la valeur estimée des paramètres de portée. Est-ce cohérent avec les résultats du modèle de régression ?

In [ ]:
library(DiceOptim)

## N.B. Il faut d'abord transformer le problème d'optimisation en minimisation 
runExperimentMin <- function(x) {
    - apply(trajectory(x), 2, max)[1]
    }
Y <- apply(X, 1, runExperimentMin)
m0 <- km(~1, design = X, response = Y)

## 1 step ##
oEGO <- max_EI(model = m0, lower = rep(0, 4), upper = rep(1, 4))
newX <- oEGO$par
newy <- runExperimentMin(newX)

cat("Expected improvement was :", round(oEGO$value,2),
    "\nActual improvement is:", round(min(Y) - newy,2),
    "\n   (>0 means the new point is better, <0 means its worst)")

In [ ]:
# On peut ensuite mettre à jour du modèle
m1 <- update(m0, newX, newy)
# et recommencer : 
## Step 2 ##
oEGO <- max_EI(model = m1, lower = rep(0, 4), upper = rep(1, 4))
newX <- oEGO$par
newy <- runExperimentMin(newX)

cat("Expected improvement was :", round(oEGO$value,2),
    "\nActual improvement is:", round(min(Y) - newy,2),
    "\n   (>0 means the new point is better, <0 means its worst)")

In [ ]:
# Heureusement, on peut faire tout automatiquement
oEGO <- EGO.nsteps(model = m0, fun = runExperimentMin, nsteps = 20, 
                   lower = rep(0, 4), upper = rep(1, 4))

bestPoint <- which.min(oEGO$value)
cat("longest shot observed:",-round(oEGO$value[bestPoint],2),
    "\ncorresponding input values:",round(oEGO$par[bestPoint,],2))


Visualisons les 20 points calculés successivement par la méthode EGO, dans le plan (X, Y), et en fonction du temps.

In [ ]:
visualizeEGO <- function(initDesign, initValues, EGOpoints, EGOvalues){
  bestIndex <- which.min(EGOvalues)
  y <- c(initValues, EGOvalues, EGOvalues[bestIndex])
  X <- rbind(initDesign, EGOpoints, EGOpoints[bestIndex, ])
  ninit <- nrow(initDesign)
  nsteps <- nrow(EGOpoints)
  pairs(cbind(y, X), 
        col = c(rep("black", ninit), rep("blue", nsteps), "red"),
        pch = c(rep(1, ninit + nsteps), 19))
}

visualizeEGO(initDesign = X, initValues = Y,
             EGOpoints = oEGO$par, EGOvalues = oEGO$val)

In [ ]:
plot(c(Y, oEGO$value), main="convergence", 
     xlab="evaluation number", ylab="Y values")
lines(rep(length(Y), 2), range(Y, oEGO$value), lty = 2, col = "gray")
lines(length(Y) + 0:length(oEGO$value), c(min(Y), cummin(oEGO$value)), col="red", lwd=2)

 Questions : 
 a) Pourquoi la méthode EGO serait beaucoup moins efficace (voire inefficace !) si on utilisait un métamodèle de régression à la place d'un processus gaussien ? 
 b) Adapter le code précédent de façon à fournir des batchs de 2 points à chaque itération (utile pour le calcul parallèle), à l'aide de la fonction qEGO.nsteps

Bonus : Comme la fonction est rapide à optimiser, on peut comparer avec le résultat d'un optimiseur standard (à choisir). On pourra utiliser la fonction suivante, qui évite la représentation graphique

In [ ]:
myFun <- function(x) apply(trajectory(x), 2, max)[1]